# Face Recognition

Muchas de las ideas presentadas aquí son de [FaceNet](https://arxiv.org/pdf/1503.03832.pdf), [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf). 

Los problemas de reconocimiento facial comúnmente se dividen en dos categorías:

- **Face verification** - "¿Es esta la persona reclamada?". Por ejemplo, en algunos aeropuertos, puede pasar por la aduana dejando que un sistema escanee su pasaporte y luego verifique que usted (la persona que lleva el pasaporte) es la persona correcta. Un teléfono móvil que se desbloquea usando tu rostro también usa verificación facial. Este es un problema de correspondencia 1:1.
- **Face Recognition** - "¿Quién es esta persona?". Por ejemplo, se muestra un [video de reconocimiento facial](https://www.youtube.com/watch?v=wr4rx0Spihs) de empleados de Baidu que ingresaban a la oficina sin necesidad de identificarse. Este es un problema de coincidencia 1:K.

FaceNet aprende una red neuronal que codifica una imagen de rostro en un vector de 128 números. Al comparar dos de estos vectores, puede determinar si dos imágenes son de la misma persona.
    
**usted:**
- Implementará la triplet loss function
- Usa un modelo previamente entrenado para mapear imágenes de rostros en codificaciones de 128 dimensiones
- Use estas codificaciones para realizar la verificación facial y el reconocimiento facial

#### Primera notación de canales

* Usaremos un modelo previamente entrenado que representa las activaciones de ConvNet usando una convención de **"channels-first"**, a diferencia de la convención de "channels last", un lote de imágenes tendrá la forma $(m, n_C, n_H, n_W)$ en lugar de $(m, n_H, n_W, n_C)$.

# **!! IMPORTANTE !!**

**<font color='red'>Debido a que el modelo es creado en un entorno con GPU, puede causar problemas a la hora de ejecutarlo en CPU</font>**

```python 
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)

  File "C:\Users\sandr\AppData\Roaming\Python\Python38\site-packages\keras\layers\pooling\base_pooling2d.py", line 84, in call
      outputs = self.pool_function(
Node: 'FaceRecoModel/max_pooling2d/MaxPool'
Default MaxPoolingOp only supports NHWC on device type CPU
	 [[{{node FaceRecoModel/max_pooling2d/MaxPool}}]] [Op:__inference_predict_function_3796]

```

**<font color='green'>En el caso de no contar con GPU, puede abrir este notebook en Google Colab, cambiar el tipo de ejecución de GPU, y luego subir los archivos</font>** `fr_utils.py`, `inception_block_v2.py` y los archivos a descargar que son [images.zip](https://drive.google.com/file/d/1u19mxnToRuOtPhLscBDJQp-POmfiCScw/view?usp=share_link) y [weights.zip](https://drive.google.com/file/d/1BBytSkyl2ckdiTWChO0uJf5gLM0ox5Ny/view?usp=share_link)


Ejecutar las siguientes celdas, una vez que tienes los archivos .zip subidos a Colab, de tal manera puedas descomprimir las carpetas

In [2]:
!unzip -q /content/images.zip -d /content/

In [3]:
!unzip -q /content/weights.zip -d /content/

replace /content/inception_blocks_v2.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


**Estructura de carpetas y archivos**

```
images <- imagenes de miembros de la compañía
weights <- Pesos pre-entrenados del modelo inception_v2 ya definido (archivos.csv)
└─── bn1_b.csv
     ...
fr_utils.py <- Funciones para la asignación de pesos a las capas del modelo y para las predicciones
inception_blocks_v2.py <- Arquitectura del modelo inception_v2 utilizado
face_recognition.ipynb <- Descripción de todo el proceso llevado en el reconocimiento facial
...

```

## 1. Importamos Librerías

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, BatchNormalization, MaxPool2D, AveragePooling2D, Dense, Flatten, Lambda
from tensorflow.keras.initializers import GlorotUniform

import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')
from fr_utils import *
from inception_blocks_v2 import *

## 2. Naive Face Verification

En Face Verification, te dan dos imágenes y tienes que determinar si son de la misma persona. La forma más sencilla de hacer esto es comparar las dos imágenes píxel por píxel. Si la distancia entre las imágenes sin procesar es menor que un umbral elegido, ¡puede ser la misma persona!

<div align='center'>
<img src="images/pixel_comparison.png" style="width:410px;height:180px;">
</div>
<caption><center> <u> <font color='blue'> Figura 1 </u></center></caption>

* Por supuesto, este algoritmo funciona muy mal, ya que los valores de los píxeles cambian drásticamente debido a las variaciones en la iluminación, la orientación del rostro de la persona, incluso cambios menores en la posición de la cabeza, etc.
* Verá que en lugar de usar la imagen sin procesar, puede aprender una codificación, $f(img)$.
* Al usar una codificación para cada imagen, una comparación de elementos produce un juicio más preciso sobre si dos imágenes son de la misma persona.

## 3. Codificación de imágenes de rostros en un vector de 128 dimensiones

### 3.1. Uso de ConvNet para calcular codificaciones

El modelo FaceNet requiere muchos datos y mucho tiempo para entrenar. Entonces, siguiendo la práctica común en el aprendizaje profundo aplicado, carguemos pesos que alguien más ya ha entrenado. La arquitectura de la red sigue el modelo Inception de [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). Hemos proporcionado una implementación de red inicial. Puede buscar en el archivo `inception_blocks_v2.py` para ver cómo se implementa

Las cosas clave que necesita saber son:

- Esta red utiliza imágenes RGB de 96x96 dimensiones como entrada. Específicamente, ingresa una imagen facial (o lote de $m$ imágenes faciales) como un tensor de forma $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$
- Produce una matriz de forma $(m, 128)$ que codifica cada imagen facial de entrada en un vector de 128 dimensiones

Ejecute la celda a continuación para crear el modelo para imágenes de rostros.

In [4]:
FRmodel = faceRecoModel(input_shape = (3,96,96))

In [5]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


Al usar una capa totalmente conectada de 128 neuronas como su última capa, el modelo garantiza que la salida sea un vector de codificación de tamaño 128. Luego, usa las codificaciones para comparar dos imágenes de rostros de la siguiente manera:

<div align='center'>
<img src="images/distance_kiank.png" style="width:680px;height:250px;">
<div>
<caption><center> <u> <font color='blue'> Figura 2: <br> </u> <font color='blue'> Al calcular la distancia entre dos codificaciones y el umbral, usted puede determinar si las dos imágenes representan a la misma persona</center></caption>

Entonces, una codificación es buena si:
- Las codificaciones de dos imágenes de la misma persona son bastante similares entre sí.
- Las codificaciones de dos imágenes de diferentes personas son muy diferentes.

La triplet loss function formaliza esto y trata de "empujar" las codificaciones de dos imágenes de la misma persona (Anchor y Positive) más juntas, mientras "jala" las codificaciones de dos imágenes de personas diferentes (Anchor, Negative) más separadas.

<div align = 'center'>
<img src="images/triplet_comparison.png" style="width:300px;height:170px;">
</div>
<br>
<caption><center> <u> <font color='blue'> Figure 3: <br> </u> <font color='blue'> En la siguiente parte, llamaremos a las imágenes de izquierda a derecha: anchor (A), positive (P), negative (N) </center></caption>

### 3.2. Triplet loss function

Para una imagen $x$, denotamos su codificación $f(x)$, donde $f$ es la función calculada por la red neuronal.

<img src="images/f_x.png" style="width:400px;height:180px;">

El entrenamiento utilizará tripletas de imágenes $(A, P, N)$:

- A es una imagen de "ancla": una imagen de una persona.
- P es una imagen "Positiva": una imagen de la misma persona que la imagen del Ancla.
- N es una imagen "negativa": una imagen de una persona diferente a la imagen de anclaje.

**Estos trillizos se seleccionan de nuestro conjunto de datos de entrenamiento**. Escribiremos $(A^{(i)}, P^{(i)}, N^{(i)})$ para indicar el ejemplo de entrenamiento $i$-ésimo.

Le gustaría asegurarse de que una imagen $A^{(i)}$ de un individuo esté más cerca de la $P^{(i)}$ positiva que de la imagen negativa $N^{(i)}$ ) por al menos un margen $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)} ) - f(N^{(i)}) \mid \mid_2^2$$

Por lo tanto, le gustaría minimizar el siguiente "triplet cost":

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i )}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Aquí, estamos usando la notación "$[z]_+$" para denotar $max(z,0)$.

Notas:
- El término (1) es la distancia al cuadrado entre el ancla "A" y el positivo "P" para un triplete dado; quieres que esto sea pequeño.
- El término (2) es la distancia al cuadrado entre el ancla "A" y la "N" negativa para un triplete dado, desea que sea relativamente grande. Tiene un signo menos que lo precede porque minimizar el negativo del término es lo mismo que maximizar ese término.
- $\alpha$ se llama margen. Es un hiperparámetro que selecciona manualmente. Usaremos $\alpha = 0.2$.

La mayoría de las implementaciones también reescalan los vectores de codificación para tener una norma L2 igual a uno (es decir, $\mid \mid f(img)\mid \mid_2$=1); no tendrás que preocuparte por eso en esta tarea.

**Ejercicio**: Implemente la pérdida de triplete como se define en la fórmula (3). Aquí están los 4 pasos:
1. Calcule la distancia entre las codificaciones de "ancla" y "positivo": $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. Calcule la distancia entre las codificaciones de "ancla" y "negativo": $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. Calcule la fórmula por ejemplo de entrenamiento: $ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^ {(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
4. Calcule la fórmula completa tomando el máximo con cero y sumando los ejemplos de entrenamiento:
$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2+ \alpha \large ] \small_+ \tag{ 3}$$

In [6]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
        Implementación de triplet loss según lo definido por la fórmula (3)

        Argumentos:
        y_true: Etiquetas verdaderas, requeridas cuando define una pérdida en Keras, no la necesita en esta función
        y_pred: Lista que contiene 3 objetos:
            * anchor: las codificaciones para las imágenes ancla de forma (None, 128)
            * positive: las codificaciones para las imágenes positivas de forma (None, 128)
            * negative: las codificaciones para las imágenes negativas de forma (None, 128)
    """

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    # step1
    post_dist = tf.reduce_sum(tf.square(anchor - positive), axis = -1)
    # step2
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis = -1)
    # step3
    basic_loss = post_dist - neg_dist + alpha

    # step4
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))

    return loss

In [7]:
tf.random.set_seed(1)
y_true = (None, None, None)
y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
          tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
          tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
loss = triplet_loss(y_true, y_pred)
print("loss = ", loss.numpy())

loss =  527.2598


### 3.3 - Cargando el modelo pre-entrenado

FaceNet se entrena minimizando la pérdida de triplete. Pero dado que el entrenamiento requiere muchos datos y muchos cálculos, no lo entrenaremos desde cero aquí. En su lugar, cargamos un modelo previamente entrenado. Cargue un modelo usando la siguiente celda; esto puede tardar un par de minutos en ejecutarse.

In [8]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

Aquí hay algunos ejemplos de distancias entre las codificaciones entre tres individuos:
<div align='center'>
<img src="images/distance_matrix.png" style="width:380px;height:200px;">
</div>
<br>
<caption><center> <u> <font color='blue'> Figura 4:</u> <br>  <font color='blue'> Ejemplo de outputs de distancia entre codificaciones de tres individuos </center></caption>

¡Usemos ahora este modelo para realizar la verificación facial y el reconocimiento facial!

Está construyendo un sistema para un edificio de oficinas en el que al administrador del edificio le gustaría ofrecer reconocimiento facial para permitir que los empleados ingresen al edificio.

Le gustaría crear un sistema de **verificación facial** que dé acceso a la lista de personas que viven o trabajan allí. Para ser admitido, cada persona debe deslizar una tarjeta de identificación (tarjeta de identificación) para identificarse en la entrada. Luego, el sistema de reconocimiento facial verifica que son quienes dicen ser.

In [9]:
FRmodel.summary()

Model: "FaceRecoModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 96, 96)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 3, 102, 102)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 64, 48, 48)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 bn1 (BatchNormalization)       (None, 64, 48, 48)   256         ['conv1[0][0]']                  
                                                                                      

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(FRmodel, show_shapes=True)

## 4 - Aplicando el modelo

### 4.1 - Face Verification

Construyamos una base de datos que contenga un vector de codificación para cada persona a la que se le permite ingresar a la oficina. Para generar la codificación usamos `img_to_encoding(image_path, model)`, que ejecuta el forward propagation del modelo en la imagen especificada.

Ejecute el siguiente código para construir la base de datos (representada como un diccionario de python). Esta base de datos asigna el nombre de cada persona a una codificación de 128 dimensiones de su rostro.

In [10]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

1/1 [==============================] - 0s 20ms/step


Ahora, cuando alguien aparece en la puerta y desliza su tarjeta de identificación (y así le da su nombre), puede buscar su codificación en la base de datos y usarla para verificar si la persona que está parada en la puerta de entrada coincide con el nombre en la identificación.

**Implemente** verify() que verifica si la imagen de la cámara de la puerta principal (`image_path`) es realmente la persona llamada "identity". Tendrás que seguir los siguientes pasos:
1. Calcule la codificación de la imagen desde `image_path`.
2. Calcular la distancia entre esta codificación y la codificación de la imagen identity almacenada en la base de datos.
3. Abra la puerta si la distancia es inferior a 0.7; de lo contrario, no la abra.

* Como se presentó anteriormente, debe usar la distancia L2 [np.linalg.norm](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html). 


In [13]:
def verify(image_path, identity, database, model):
    """
     Función que verifica si la persona en la imagen "image_path" es "identity".
    
     Argumentos:
     image_path -- ruta a una imagen
     identity: string, nombre de la persona cuya identidad desea verificar. Tiene que ser un empleado que trabaja en la oficina.
     database: el diccionario de python asigna nombres de personas permitidas (strings) a sus codificaciones (vectores).
     model: su instancia de modelo Inception en Keras
    
     Devoluciones:
     dist: distancia entre image_path y la imagen de "identity" en la base de datos.
     door_open -- True, si la puerta debe abrirse. False en caso contrario.
     """
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding - database[identity])

    if dist < 0.7:
        print("Usted " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("Usted no es " + str(identity) + ", por favor siga adelante")
        door_open = False

    return dist, door_open



Younes está tratando de entrar a la oficina y la cámara le toma una foto ("images/camera_0.jpg"). Ejecutemos su algoritmo de verificación en esta imagen:

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [14]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

1/1 [==============================] - 0s 28ms/step
Usted younes, welcome in!


(0.66713977, True)

Benoit, que no trabaja en la oficina, robó la tarjeta de identificación de Kian e intentó ingresar a la oficina. La cámara tomó una foto de Benoit ("images/camera_2.jpg). Ejecutemos el algoritmo de verificación para verificar si benoit puede ingresar.

<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [15]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

1/1 [==============================] - 0s 22ms/step
Usted no es kian, por favor siga adelante


(0.85868865, False)

### 4.2 - Face Recognition

Su sistema de verificación facial funciona bien en su mayoría. Pero como a Kian le robaron su tarjeta de identificación, cuando regresó a la oficina al día siguiente ¡no pudo entrar!

Para resolver esto, le gustaría cambiar su sistema de verificación facial a un sistema de reconocimiento facial. De esta manera, ya nadie tiene que llevar una tarjeta de identificación. ¡Una persona autorizada puede caminar hasta el edificio y la puerta se desbloqueará para ellos!

Implementará un sistema de reconocimiento facial que toma como entrada una imagen y determina si es una de las personas autorizadas (y, de ser así, quién). A diferencia del sistema de verificación facial anterior, ya no obtendremos el nombre de una persona como una de las entradas.


**Implementar** `who_is_it()`. Tendrás que seguir los siguientes pasos:
1. Calcule la codificación de destino de la imagen de image_path
2. Encuentre la codificación de la base de datos que tiene la distancia más pequeña con la codificación de destino.
     - Inicializar la variable `min_dist` a un número lo suficientemente grande (100). Le ayudará a realizar un seguimiento de cuál es la codificación más cercana a la codificación de la entrada.
     - Bucle sobre los nombres y codificaciones del diccionario de la base de datos. Para hacer un bucle, use `for (name, db_enc) en base de datos.items()`.
         - Calcule la distancia L2 entre la "codificación" de destino y la "codificación" actual de la base de datos.
         - Si esta distancia es menor que min_dist, establezca `min_dist` en `dist` e `identity` en `name`.

In [16]:
def who_is_it(image_path, database, model):
    """
     Implementa el reconocimiento facial para la oficina al encontrar quién es la persona en la imagen image_path.
    
     Argumentos:
     image_path -- ruta a una imagen
     database: base de datos que contiene codificaciones de imágenes junto con el nombre de la persona en la imagen
     model: su instancia de modelo Inception en Keras
    
     Devoluciones:
     min_dist: la distancia mínima entre la codificación image_path y las codificaciones de la base de datos
     identity-- string, la predicción de nombre para la persona en image_path
     """

    encoding = img_to_encoding(image_path, model)
    # Inicializar "min_dist" a un valor grande, digamos 100
    min_dist = 100
    # Recorre los nombres y codificaciones del diccionario de la base de datos.
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding-db_enc)
        # Si esta distancia es menor que min_dist, establezca min_dist en dist e identidad en name. (≈ 3 líneas)
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.7:
            print('No está en la base de datos')
    else:
            print("Está " + str(identity) + ", la distancia es " + str(min_dist))
            
    return min_dist, identity


Younes está en la puerta principal y la cámara le toma una foto ("images/camera_0.jpg"). Veamos si su algoritmo who_it_is() identifica a Younes.

In [17]:
who_is_it("images/camera_0.jpg", database, FRmodel)

1/1 [==============================] - 0s 20ms/step
Está younes, la distancia es 0.66713977


(0.66713977, 'younes')

#### Formas de mejorar su modelo de reconocimiento facial
Aunque no lo implementaremos aquí, aquí hay algunas formas de mejorar aún más el algoritmo:
- Poner más imágenes de cada persona (en diferentes condiciones de iluminación, tomadas en diferentes días, etc.) en la base de datos. Luego, dada una nueva imagen, compare la nueva cara con varias imágenes de la persona. Esto aumentaría la precisión.
- Recorte las imágenes para que solo contengan la cara y menos la región del "borde" alrededor de la cara. Este preprocesamiento elimina algunos de los píxeles irrelevantes alrededor de la cara y también hace que el algoritmo sea más robusto.

## Puntos clave para recordar
- La verificación facial resuelve un problema de coincidencia 1:1 más fácil; el reconocimiento facial aborda un problema de coincidencia 1:K más difícil.
- La pérdida de triplete es una función de pérdida efectiva para entrenar una red neuronal para aprender una codificación de una imagen de rostro.
- La misma codificación se puede utilizar para la verificación y el reconocimiento. Medir las distancias entre las codificaciones de dos imágenes le permite determinar si son fotografías de la misma persona.

#### Referencias

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- The pretrained model we use is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 